# Lab 2: pH and Properties of Buffers

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt

from scipy.interpolate import BPoly
from matplotlib.patches import Rectangle
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Enzyme activity

In [2]:
# Read the csv
EA = pd.read_csv('/Users/kev/Documents/Python/BCMB-301A--Lab Techniques/Data/Lab 2/pH on Enzyme.csv', 
                 header=2)

EA = EA.set_index('Time (s)')

In [3]:
# Fine the derivative 
EA_d = EA.diff()
EA_d = EA_d.rename(index=str, columns={"3":"d3", "5":"d5", "7.5":"d7.5", "9":"d9", "10":"d10"})

In [4]:
# Concat the data with the derivative (buggy)
EA_conc = pd.concat([EA, EA_d], axis=1)

In [5]:
EA

,3,5,7.5,9,10
Time (s),,,,,
0,0.000,0.000,0.000,0.000,0.000
10,0.001,-0.096,-0.166,0.024,0.010
20,0.001,-0.155,-0.142,0.036,0.035
30,0.003,-0.146,-0.115,0.047,0.048
40,0.001,0.014,-0.090,0.060,0.068
50,0.000,0.087,-0.063,0.074,0.088
60,0.002,0.102,-0.037,0.088,0.105
70,0.001,0.104,-0.012,0.102,0.123
80,0.000,0.104,0.015,0.115,0.146


In [6]:
EA_d

,d3,d5,d7.5,d9,d10
Time (s),,,,,
0,NaN,NaN,NaN,NaN,NaN
10,0.001,-0.096,-0.166,0.024,0.010
20,0.000,-0.059,0.024,0.012,0.025
30,0.002,0.009,0.027,0.011,0.013
40,-0.002,0.160,0.025,0.013,0.020
50,-0.001,0.073,0.027,0.014,0.020
60,0.002,0.015,0.026,0.014,0.017
70,-0.001,0.002,0.025,0.014,0.018
80,-0.001,0.000,0.027,0.013,0.023


In [7]:
# Average enzyme activity 
EA_d_ave = EA_d.iloc[2:,:].mean(axis=0)
EA_d_ave

d3     -0.000091
d5      0.020818
d7.5    0.026364
d9      0.013636
d10     0.018727
dtype: float64

In [8]:
# Rate per min 
EA_d_min = EA_d_ave * 6
EA_d_min

d3     -0.000545
d5      0.124909
d7.5    0.158182
d9      0.081818
d10     0.112364
dtype: float64

In [9]:
# Relative enzyme activity
EA_d_ave_percent = EA_d_ave / np.max(EA_d_ave) * 100
EA_d_ave_percent

d3       -0.344828
d5       78.965517
d7.5    100.000000
d9       51.724138
d10      71.034483
dtype: float64

In [10]:
plt.figure(figsize=(6, 4))
# EA_d_ave_percent.plot(kind='line')
# EA_d_ave_percent.plot(kind='scatter')
plt.plot(list(EA), EA_d_ave_percent)
plt.scatter(list(EA), EA_d_ave_percent, s=5, color='orange', zorder=5)
plt.title("Percent enzyme activity vs pH")
plt.xlabel('pH')
plt.ylabel('Relative enzyme activity (%)')
plt.grid()

<Figure size 432x288 with 1 Axes>

## Titration curve: Cubic spline fit

In [11]:
x = [-0.5, 0.5, 1.5]
x1 = [0.5, 1.5, 2.5]
y = [2.35, 7.63, 10.52]

In [12]:
def tit_graph(x, y):    
    Tit = BPoly.from_derivatives([x[0]-0.5, x[0], x[1], x[2], x[2]+0.5], 
                                  [[y[0]-0.5*(y[1]-y[0])], [y[0], 0, 0, 0, 1], 
                                   [y[1], 0, 0, 0, 1], [y[2], 0, 0, 0, 1], [y[2]+0.5*(y[2]-y[1])]])

    Tit_space = np.linspace(x[0]-0.5, x[2]+0.5, 300)

    pKa = [r'$\mathrm{{pK_{{a(COOH)}}^1}} =$ {0}'.format(y[0]), 
           r'$\mathrm{{pK_{{a(NH_3^+)}}^2}} =$ {0}'.format(y[1]), 
           r'$\mathrm{{pK_{{a(NH_3^+)}}^3}} =$ {0}'.format(y[2])]
    
    reg = ['Buffer region 1', 'Buffer region 2', 'Buffer region 3']

    fig, ax = plt.subplots()
    ax.plot(Tit_space, Tit(Tit_space))
    ax.scatter(x, y, s=5, color='orange', zorder=5)

    # Draw boxes 
    w0, w1, w2 = 0.5, 0.5, 0.6
    h = 0.5
    r1 = Rectangle((x[0]-0.5*w0, y[0]-0.5*h), w0, h, angle=0)
    r2 = Rectangle((x[1]-0.5*w1, y[1]-0.5*h), w1, h, angle=0)
    r3 = Rectangle((x[2]-0.5*w2, y[2]-0.5*h), w2, h, angle=0)
    bx = [r1, r2, r3]
    pc = mp.collections.PatchCollection(bx, color='g', alpha=0.25)
    ax.add_collection(pc)

    # Label regions
    for i, txt in enumerate(pKa):
        ax.annotate(txt, (x[i]-0.3, y[i]-1), fontsize=8)

    for i, txt in enumerate(reg):
        ax.annotate(txt, (x[i]-0.3, y[i]+0.5), fontsize=8)


    # Arrows in xlabel
    if np.min(x)<0 and np.max(x)>0: 
        ax.annotate('', xy=(np.max(x)+0.5, -2.5), xytext=(0.05, -2.5), #draws an arrow from one set of coordinates to the other
                    arrowprops=dict(arrowstyle='->'),     #sets style of arrow and colour
                    annotation_clip=False)                #This enables the arrow to be outside of the plot

        ax.annotate('', xy=(np.min(x)-0.5, -2.5), xytext=(-0.05, -2.5),
                    arrowprops=dict(arrowstyle='->'),     
                    annotation_clip=False)  

    #     Label x-axis
        ax.annotate(r'Equvalent $[\mathrm{OH^-}]$ added',
                    xy=(0, 0), xytext=(0.15, -3.5),
                    annotation_clip=False)

        ax.annotate(r'Equvalent $[\mathrm{H^+}]$ added', 
                    xy=(0, 0), xytext=(-1.25, -3.5),
                    annotation_clip=False)
        
        ab = r'acid $[\mathrm{{H^+}}]$ or base $[\mathrm{{OH^-}}]$'

    elif np.min(x) > 0: 
        plt.xlabel(r'Equvalent $[\mathrm{OH^-}]$ added')
        ab = r'base $[\mathrm{{OH^-}}]$'

    elif np.min(x)<0 and np.max(x)<0:
        plt.xlabel(r'Equvalent $[\mathrm{H^+}]$ added')
        ab = r'acid $[\mathrm{{H^+}}]$'

    # plt.xlabel('Charge?')
    tit_title = r'Changes in solution pH with Buffer X with introduction of molar equivalents of {0}'.format(ab)
    plt.title("\n".join(wrap(tit_title, 70)))
    plt.ylabel('pH')
    plt.grid()
    plt.show()

In [13]:
tit_graph(x, y)

<Figure size 432x288 with 1 Axes>

In [14]:
tit_graph(x1, y)

<Figure size 432x288 with 1 Axes>